In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [61]:
DATA_FOLDER = './readonly/'

data = pd.read_excel(os.path.join(DATA_FOLDER, 'MFTRAVEL_HACKATHON.xlsx'), sheet_name=None)
countries = data['CountryDict']['CNTRY_CODE'].values.astype('int32')

In [283]:
np.savetxt('countries.txt', countries, fmt='%d')
c = np.loadtxt('countries.txt', dtype=int)

## Recomendation system

In [3]:
import tensorflow as tf
from sklearn import preprocessing

In [62]:
exp = data['Data'].copy()

def foo(x):
    if type(x) == int:
        return [x]
    return list(map(int, x.split('; ')))

exp['COUNTRIES_IN_TRIP'] = exp['COUNTRIES_IN_TRIP'].apply(foo)

In [63]:
exp['USING_INTERNET'] = exp['USING_INTERNET'].fillna(0).astype('bool')
le = preprocessing.LabelEncoder()

for column in ['GENDER', 'REGION', 'DEVICE_TYPE',
               'OS', 'SUBSAGE_MF_SEGMENT']:
    le.fit(list(exp[column].values))
    exp[column] = le.transform(list(exp[column].values))
    print('OKAY', column)

OKAY GENDER
OKAY REGION
OKAY DEVICE_TYPE
OKAY OS
OKAY SUBSAGE_MF_SEGMENT


In [64]:
# IT'S HEAVY CALCULATION
visited_country = dict()
for i in countries:
    list_t = []
    for j in exp['COUNTRIES_IN_TRIP']:
        list_t.append(i in j)
    
    visited_country[i] = list_t
    
exp = exp.join(pd.DataFrame(visited_country))
exp.drop(['COUNTRIES_IN_TRIP'], inplace = True, axis=1)

exp['User_ID'] = exp['User_ID'].astype('int32')
exp['AGE'] = exp['AGE'].fillna(-1).astype('int8')
exp['GENDER'] = exp['GENDER'].astype('int8')
exp['REGION'] = exp['REGION'].astype('int8')

In [246]:
exp.to_csv('exp.csv')

In [65]:
tempo = exp.groupby(['User_ID']).agg({'SMS_IN_CNT_M3' : ['mean', 'std'], 
                                      'TRIP_DURATION' : ['mean', 'std'],
                                      'MOU_IN_REVENUE_M3' : ['mean', 'std'],
                                      'MOU_OUT_REVENUE_M3' : ['mean', 'std'], 
                                      'DOU_DURATION_M3' : ['mean', 'std'], 
                                      'ARPU_M3' : ['mean', 'std']
                                     })
foo = lambda x : "_".join(x) if x[1]!="" else x[0]
tempo.columns = [foo(x) for x in tempo.columns.ravel()]

user_to_flights = exp.groupby(['User_ID'])[exp.columns[18:]].sum().values
user_to_mostfreq_country = countries[user_to_flights.argmax(axis=1)]

tempo['most_freq'] = user_to_mostfreq_country
user_data = tempo.join(exp.groupby(['User_ID']).agg({'GENDER' : 'max', 'AGE' : 'max', 'REGION' : 'max',
                              'DEVICE_TYPE' : 'max', 'OS' : 'max', 'SUBSAGE_MF_SEGMENT' : 'max', 
                              'USING_INTERNET' : 'max'}))
del tempo

dct = dict()
for i in countries:
    dct[i] = 'sum'

user_data = user_data.join(exp.groupby(['User_ID']).agg(dct))
user_data.fillna(0, inplace=True)

In [247]:
user_data.to_csv('user_data.csv')

In [261]:
true_data.to_csv('true_data.csv')

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Reshape, Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import LeakyReLU, Dropout, ReLU, BatchNormalization, InputLayer, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop

In [271]:
dropout = 0.2

inp = tf.keras.layers.Input((21 + 245,))

k = Dense(256)(inp)
k = LeakyReLU(alpha=0.2)(k)
k = Dropout(dropout)(k)

k = Dense(128)(k)
k = LeakyReLU(alpha=0.2)(k)
k = Dropout(dropout)(k)

k = Dense(64)(k)
k = LeakyReLU(alpha=0.2)(k)
k = Dropout(dropout)(k)

k = Dense(32)(k)
k = LeakyReLU(alpha=0.2)(k)
k = Dropout(dropout)(k)

k = Dense(16)(k)
k = LeakyReLU(alpha=0.2)(k)
k = Dropout(dropout)(k)

k = Dense(1)(k)

D = tf.keras.Model(
            inputs=inp, 
            outputs=Activation('sigmoid')(k))

optimizer = RMSprop(lr=0.0002, decay=6e-8)
D.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [272]:
dropout = 0

inputs_ = tf.keras.layers.Input((20 + 245,))

dense_1 = Dense(129)(inputs_)
nonlin_1 = LeakyReLU(alpha=0.2)(dense_1)
drop_1 = Dropout(dropout)(nonlin_1)

dense_21 = Dense(64)(drop_1)
nonlin_21 = LeakyReLU(alpha=0.2)(dense_21)
drop_21 = Dropout(dropout)(nonlin_21)

dense_22 = Dense(32)(drop_21)
nonlin_22 = LeakyReLU(alpha=0.2)(dense_22)
drop_22 = Dropout(dropout)(nonlin_22)

dense_3 = Dense(8)(drop_22)
nonlin_3 = LeakyReLU(alpha=0.2)(dense_3)
drop_3 = Dropout(dropout)(nonlin_3)

predictions = Dense(1)(drop_3)

conc = tf.concat(axis=1, values=[inputs_, predictions])

optimizer = Adam(1e-6)
G = tf.keras.Model(
            inputs=inputs_, 
            outputs=conc)

def loss(y, y_pred):
    return tf.reduce_mean((y_pred[:, -1:]-y[:, -1:])**2)

G.compile(loss=loss, optimizer=optimizer)

res_D = D(conc)

AM =  tf.keras.Model(
            inputs=inputs_, 
            outputs=res_D)
optimizer = RMSprop(lr=0.0001, decay=3e-8)
AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [275]:
exp[['User_ID', 'TRIP_MAIN_COUNTRY']]

,User_ID,TRIP_MAIN_COUNTRY
0,80000,222.0
1,80000,85.0
2,80001,213.0
3,80001,23.0
4,80001,202.0
...,...,...
378462,123215,19.0
378463,177003,175.0
378464,180451,1.0
378465,156886,12.0


In [276]:
user_data

,SMS_IN_CNT_M3_mean,SMS_IN_CNT_M3_std,TRIP_DURATION_mean,TRIP_DURATION_std,MOU_IN_REVENUE_M3_mean,MOU_IN_REVENUE_M3_std,MOU_OUT_REVENUE_M3_mean,MOU_OUT_REVENUE_M3_std,DOU_DURATION_M3_mean,DOU_DURATION_M3_std,...,860,548,336,862,704,876,732,887,894,716
User_ID,,,,,,,,,,,,,,,,,,,,,
80000,299.000000,1.154701,4.500,1.290994,532.116667,78.731334,466.699996,67.222821,5381.978539,1291.962179,...,False,False,False,False,False,False,False,False,False,False
80001,395.050000,143.792788,4.650,4.749238,250.322501,107.936626,301.601666,79.642260,1857.871891,1692.410125,...,False,False,False,False,False,False,False,False,False,False
80002,164.500000,28.991378,7.000,8.485281,84.649999,19.963982,78.641666,15.026017,8816.631841,3352.524450,...,False,False,False,False,False,False,False,False,False,False
80003,249.666667,63.006878,5.500,4.929503,822.913890,417.014447,628.141667,537.880583,3695.561045,843.805226,...,False,False,False,False,False,False,False,False,False,False
80004,438.625000,109.247998,3.625,4.274091,295.881251,82.766410,353.275003,118.160949,2520.610484,962.627535,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180448,176.000000,0.000000,4.000,0.000000,173.050002,0.000000,280.716670,0.000000,1273.079104,0.000000,...,False,False,False,False,False,False,False,False,False,False
180449,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
180450,0.000000,0.000000,11.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False


In [274]:
ids_list = user_data.index.unique()

epochs = 50
batch_size_noise = 64
batch_size_true = 128

true_data = exp[['User_ID', 'TRIP_MAIN_COUNTRY']].join(user_data, on = 'User_ID')

cols = true_data.columns.tolist()
true_data = true_data[cols[0:1] + cols[2:] + cols[1:2] ]

In [273]:
for j in range(100):
        r_f = np.random.choice(true_data.shape[0], 
                                            batch_size_true, 
                                            replace=False)
        true_flights = true_data.loc[r_f].values
        us_id = true_data.loc[r_f]['User_ID'].values
        fake_batch = user_data.loc[us_id].values
        
        G.fit(fake_batch.astype('float32'), true_flights.astype('float32'))

Train on 128 samples
128/128 [==============================] - 1s 8ms/sample - loss: 1337565.9766
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 571565.4922
Train on 128 samples
128/128 [==============================] - 1s 4ms/sample - loss: 330530.3984
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 313745.6172
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 357174.4297
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 658137.4922
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 592893.0938
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 510939.0156
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 2035069.3125
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 752075.8867
Train on 128 sampl

In [266]:
for j in range(50):
    true_flights = true_data.loc[np.random.choice(true_data.shape[0], 
                                            batch_size_true, 
                                            replace=False)].drop(['User_ID'], axis=1).values
    
    random_ids = np.random.choice(ids_list, batch_size_noise, replace=False)
    fake_batch = user_data.loc[random_ids].values
    fake_flights = G.predict(fake_batch.astype('float32'))

    x = np.concatenate((true_flights, fake_flights))
    y = np.ones([batch_size_true + batch_size_noise, 1])
    y[batch_size_true:, :] = 0
    
    d_loss = D.fit(x.astype('float32'), y.astype('float32'))

Train on 192 samples
192/192 [==============================] - 2s 10ms/sample - loss: 106.6215 - accuracy: 0.5156
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 78.2483 - accuracy: 0.5729
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 60.9317 - accuracy: 0.5625
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 48.3329 - accuracy: 0.6250
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 50.6116 - accuracy: 0.6042
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 69.8460 - accuracy: 0.5156
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 43.0728 - accuracy: 0.6042
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 40.8886 - accuracy: 0.5833
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 41.8310 

In [267]:
for i in tqdm_notebook(range(epochs*100)):
    for j in range(3):
        true_flights = true_data.loc[np.random.choice(true_data.shape[0], 
                                            batch_size_true, 
                                            replace=False)].drop(['User_ID'], axis=1).values
        
        random_ids = np.random.choice(ids_list, batch_size_noise, replace=False)
        fake_batch = user_data.loc[random_ids].values
        fake_flights = G.predict(fake_batch.astype('float32'))

        x = np.concatenate((true_flights, fake_flights))
        y = np.ones([batch_size_true + batch_size_noise, 1])
        y[batch_size_true:] = 0
        
        d_loss = D.fit(x.astype('float32'), y.astype('float32'))
    
    D.trainable = False
    for j in range(5):
        random_ids = np.random.choice(ids_list, batch_size_noise, replace=False)
        fake_batch = user_data.loc[random_ids].values

        y = np.ones([batch_size_noise, 1])
        
        a_loss = AM.fit(fake_batch.astype('float32'), y.astype('float32'))
    
    D.trainable = True

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 28.8765 - accuracy: 0.5885
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 14.7689 - accuracy: 0.5729
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 16.2327 - accuracy: 0.5729
Train on 64 samples
64/64 [==============================] - 1s 23ms/sample - loss: 11.5674 - accuracy: 0.5781
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 9.4338 - accuracy: 0.6406
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 15.2594 - accuracy: 0.7188
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 7.3599 - accuracy: 0.7344
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 4.3039 - accuracy: 0.8281
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 14.0871 - accuracy: 0.6146


192/192 [==============================] - 1s 4ms/sample - loss: 12.3545 - accuracy: 0.5104
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 7.6539 - accuracy: 0.7188
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 5.2060 - accuracy: 0.7500
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 2.9501 - accuracy: 0.7500
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 3.6618 - accuracy: 0.7188
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 5.4422 - accuracy: 0.7188
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 18.1170 - accuracy: 0.6250
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 12.9241 - accuracy: 0.6042
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 11.0394 - accuracy: 0.6094
Train on 64 samples
64/6

64/64 [==============================] - 0s 6ms/sample - loss: 3.0518 - accuracy: 0.7656
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 4.0277 - accuracy: 0.7188
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 5.2605 - accuracy: 0.7344
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 12.9918 - accuracy: 0.8125
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 7.9109 - accuracy: 0.5990
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 9.5262 - accuracy: 0.5729
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 9.1063 - accuracy: 0.5469
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 3.1918 - accuracy: 0.6875
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 3.9160 - accuracy: 0.7188
Train on 64 samples
64/64 [==

192/192 [==============================] - 1s 4ms/sample - loss: 5.0580 - accuracy: 0.6250
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 3.3508 - accuracy: 0.5990
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 11.3684 - accuracy: 0.5729
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.3848 - accuracy: 0.6406
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7535 - accuracy: 0.7188
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 1.6132 - accuracy: 0.7656
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.6753 - accuracy: 0.7656
Train on 64 samples
64/64 [==============================] - 0s 6ms/sample - loss: 1.5867 - accuracy: 0.7344
Train on 192 samples
192/192 [==============================] - 1s 4ms/sample - loss: 5.7243 - accuracy: 0.5833
Train on 192 samples
192/19

...

192/192 [==============================] - 2s 8ms/sample - loss: 0.6381 - accuracy: 0.6667
Train on 64 samples
64/64 [==============================] - 1s 16ms/sample - loss: 0.4247 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.4353 - accuracy: 0.9688
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.4020 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 16ms/sample - loss: 0.3546 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.2952 - accuracy: 1.0000
Train on 192 samples
192/192 [==============================] - 2s 8ms/sample - loss: 0.6655 - accuracy: 0.6667
Train on 192 samples
192/192 [==============================] - 2s 8ms/sample - loss: 0.6847 - accuracy: 0.6667
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.5763 - accuracy: 0.6615
Train on 64 samples
64/

64/64 [==============================] - 1s 16ms/sample - loss: 0.3617 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.3820 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.3187 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.3447 - accuracy: 1.0000
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.6543 - accuracy: 0.6719
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.6364 - accuracy: 0.6667
Train on 192 samples
192/192 [==============================] - 2s 8ms/sample - loss: 0.6380 - accuracy: 0.6667
Train on 64 samples
64/64 [==============================] - 1s 15ms/sample - loss: 0.4448 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 15ms/sample - loss: 0.3879 - accuracy: 0.9844
Train on 64 samples
64/6

64/64 [==============================] - 1s 14ms/sample - loss: 0.3669 - accuracy: 0.9844
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.2925 - accuracy: 1.0000
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.7035 - accuracy: 0.6562
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.6152 - accuracy: 0.6719
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.6140 - accuracy: 0.6510
Train on 64 samples
64/64 [==============================] - 1s 15ms/sample - loss: 0.4181 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 14ms/sample - loss: 0.3883 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 15ms/sample - loss: 0.3420 - accuracy: 0.9844
Train on 64 samples
64/64 [==============================] - 1s 15ms/sample - loss: 0.3171 - accuracy: 1.0000
Train on 64 samples
64/6

192/192 [==============================] - 2s 10ms/sample - loss: 0.8620 - accuracy: 0.6667
Train on 192 samples
192/192 [==============================] - 2s 9ms/sample - loss: 0.6293 - accuracy: 0.6562
Train on 192 samples
192/192 [==============================] - 2s 10ms/sample - loss: 0.6432 - accuracy: 0.6667
Train on 64 samples
64/64 [==============================] - 1s 18ms/sample - loss: 0.4072 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 19ms/sample - loss: 0.4042 - accuracy: 1.0000
Train on 64 samples
64/64 [==============================] - 1s 17ms/sample - loss: 0.3324 - accuracy: 1.0000


KeyboardInterrupt: 

In [268]:
pd.DataFrame(G.predict(fake_batch.astype('float32')))

,0,1,2,3,4,5,6,7,8,9,...,256,257,258,259,260,261,262,263,264,265
0,344.000000,0.000000,2.000000,1.732051,506.083344,0.000000,627.783325,0.000000,30138.779297,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2153.694580
1,805.727295,174.333633,5.272727,3.770700,305.049988,245.568542,790.762146,360.099701,35645.050781,22665.033203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1743.589355
2,383.000000,0.000000,4.000000,1.414214,235.383331,0.000000,786.950012,0.000000,11747.606445,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1576.407227
3,115.000000,0.000000,1.000000,0.000000,21.233332,0.000000,43.333332,0.000000,1951.019531,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,833.664612
4,260.000000,0.000000,1.000000,0.000000,186.150009,0.000000,144.833328,0.000000,48767.707031,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3870.612793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,89.000000,0.000000,10.000000,0.000000,372.649994,0.000000,406.166656,0.000000,2242.154297,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,898.164001
60,397.000000,0.000000,3.500000,3.535534,149.583328,0.000000,264.666656,0.000000,4236.834961,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877.059326
61,85.000000,0.000000,1.500000,0.707107,3.333333,0.000000,90.033333,0.000000,5021.670898,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,896.393127
62,175.000000,0.000000,1.000000,0.000000,60.816666,0.000000,178.100006,0.000000,17000.257812,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1645.224365


In [269]:
true_data

,User_ID,SMS_IN_CNT_M3_mean,SMS_IN_CNT_M3_std,TRIP_DURATION_mean,TRIP_DURATION_std,MOU_IN_REVENUE_M3_mean,MOU_IN_REVENUE_M3_std,MOU_OUT_REVENUE_M3_mean,MOU_OUT_REVENUE_M3_std,DOU_DURATION_M3_mean,...,548,336,862,704,876,732,887,894,716,TRIP_MAIN_COUNTRY
0,80000,299.000000,1.154701,4.500000,1.290994,532.116667,78.731334,466.699996,67.222821,5381.978539,...,False,False,False,False,False,False,False,False,False,222.0
1,80000,299.000000,1.154701,4.500000,1.290994,532.116667,78.731334,466.699996,67.222821,5381.978539,...,False,False,False,False,False,False,False,False,False,85.0
2,80001,395.050000,143.792788,4.650000,4.749238,250.322501,107.936626,301.601666,79.642260,1857.871891,...,False,False,False,False,False,False,False,False,False,213.0
3,80001,395.050000,143.792788,4.650000,4.749238,250.322501,107.936626,301.601666,79.642260,1857.871891,...,False,False,False,False,False,False,False,False,False,23.0
4,80001,395.050000,143.792788,4.650000,4.749238,250.322501,107.936626,301.601666,79.642260,1857.871891,...,False,False,False,False,False,False,False,False,False,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378462,123215,141.714286,179.504609,3.285714,2.984085,50.095238,63.497327,53.183333,67.300009,3864.477401,...,False,False,False,False,False,False,False,False,False,19.0
378463,177003,58.000000,50.229473,5.333333,6.658328,241.600000,209.231738,163.577776,141.662510,6933.653649,...,False,False,False,False,False,False,False,False,False,175.0
378464,180451,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,1.0
378465,156886,35.333333,18.230012,12.666667,2.081666,192.750000,163.797605,419.761113,214.269399,1033.710614,...,False,False,False,False,False,False,False,False,False,12.0


In [236]:
toto = G.predict(user_data)[:, -1:]

In [237]:
toto[toto.astype('int32').reshape(-1) == 40]

array([[40.952805],
       [40.3265  ],
       [40.7912  ],
       [40.600132],
       [40.338852],
       [40.64759 ],
       [40.363457],
       [40.398113],
       [40.866573],
       [40.68617 ],
       [40.764164],
       [40.60462 ],
       [40.660133],
       [40.831135],
       [40.957203],
       [40.580338],
       [40.329247],
       [40.869366],
       [40.765163],
       [40.272343],
       [40.429474],
       [40.08155 ],
       [40.38868 ],
       [40.254807],
       [40.056915],
       [40.961414],
       [40.363728],
       [40.35913 ],
       [40.352924],
       [40.284004],
       [40.191452],
       [40.39775 ],
       [40.07972 ],
       [40.832394],
       [40.522495],
       [40.10779 ],
       [40.65917 ],
       [40.196754],
       [40.0724  ],
       [40.376434],
       [40.381493],
       [40.064396],
       [40.79233 ],
       [40.16374 ],
       [40.39046 ],
       [40.03674 ],
       [40.57522 ],
       [40.104046]], dtype=float32)

In [ ]:
G.save_weights('./checkpoints/my_checkpointG')
G.load_weights('./checkpoints/my_checkpointG')

D.save_weights('./checkpoints/my_checkpointD')
D.load_weights('./checkpoints/my_checkpointD')

AM.save_weights('./checkpoints/my_checkpointAM')
AM.load_weights('./checkpoints/my_checkpointAM')

## Parcing

In [105]:
import requests
import re

from bs4 import BeautifulSoup

In [238]:
lol = dict()
ban_list = []
for country_name in data['CountryDict']['NAME_R']:
    orig = country_name
    country_name = country_name.split()
    
    cc = []
    for i in country_name:
        if i == 'о-ва':
            cc.append('острова')
        elif i == 'о-в':
            cc.append('остров') 
        elif i[0]!='(':
            cc.append(i)
        
    country_name = '_'.join(cc)
    
    page = requests.get('https://ru.wikipedia.org/wiki/{}'.format(country_name))
    soup = BeautifulSoup(page.text, 'html.parser')
    columns = soup.findAll(attrs = {'data-wikidata-property-id' : "P36"})
    if columns == []:
        ban_list.append(orig)
    else:
        try:
            lol[orig] = columns[0].find('a').text
        except AttributeError:
            lol[orig] = columns[0].text

In [239]:
cities = data['CountryDict'].copy()
cities['CAPITAL'] = cities['NAME_R'].apply(lambda x : x if x in ban_list else lol[x])

In [241]:
lat = dict()
lon = dict()
ban_list = []
for name in cities['CAPITAL']:
    if name == None:
        continue
    
    orig = name
    name = name.split()
    
    cc = []
    for i in name:
        if i == 'о-ва':
            cc.append('острова')
        elif i == 'о-в':
            cc.append('остров') 
        elif i[0]!='(':
            cc.append(i)
        
    name = '_'.join(cc)
    
    page = requests.get('https://ru.wikipedia.org/wiki/{}'.format(name))
    soup = BeautifulSoup(page.text, 'html.parser')
    columns = soup.findAll('span', attrs = {'title' : "Показать карту"})
    
    if columns == []:
        ban_list.append(orig)
        continue
        
    r = columns[0]
    lon[orig] = r.find_all('a')[0].get('data-lon')
    lat[orig] = r.find_all('a')[0].get('data-lat')

In [245]:
ban_list.append(None)
cities['LAT'] = cities['CAPITAL'].apply(lambda x : None if x in ban_list else lat[x])
cities['LON'] = cities['CAPITAL'].apply(lambda x : None if x in ban_list else lon[x])
cities

,CNTRY_CODE,NAME,NAME_R,CAPITAL,LAT,LON
0,895.0,Abkhazia,Абхазия,None,None,None
1,4.0,Afghanistan,Афганистан,Кабул,34.53278,69.16583
2,8.0,Albania (Republic of),Албания,Тирана,41.33,19.82
3,12.0,Algeria (People`s Democratic Republic of),Алжир,Алжир,27.25,3.45
4,16.0,American Samoa,Американское (Восточное) Самоа,Паго-Паго,-14.27306,-170.70278
...,...,...,...,...,...,...
242,732.0,Western Sahara,Западная Сахара,None,None,None
243,887.0,Yemen (Republic of),Йемен,Сана,15.35,44.2
244,894.0,Zambia (Republic of),Замбия,Лусака,-15.41667,28.28333
245,716.0,Zimbabwe (Republic of),Зимбабве,Хараре,-17.86389,31.02972


In [270]:
cities.to_csv('cities.csv')